In [1]:
from scipy import stats as st
import numpy as np
import pandas as pd
import os
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
!pip install --upgrade xlrd
!pip install umap-learn
!pip install kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 2.2 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 2.9 MB/s 
     |████████████████████████████████| 1.1 MB 32.5 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=9abd754468dcc27f9944f6064e13d47d2f516c2b8d73c2f7a794a94827945134
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=2c70dc1f024e940a681fe617aa4fb453ddb6a4f5669d2ec5dad0f4220282dc4e
  Stored in directory: /root/.cache/pip/wheels/7f/2a/f8/7bd5dce

In [3]:
import zipfile
!rm Bin_data.zip
!wget https://github.com/vitvara/tobacco_project_review/raw/main/zip_file/Bin_data.zip
with zipfile.ZipFile("Bin_data.zip","r") as zip_ref:
    zip_ref.extractall()

rm: cannot remove 'Bin_data.zip': No such file or directory
--2022-06-07 16:52:34--  https://github.com/vitvara/tobacco_project_review/raw/main/zip_file/Bin_data.zip
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vitvara/tobacco_project_review/main/zip_file/Bin_data.zip [following]
--2022-06-07 16:52:34--  https://raw.githubusercontent.com/vitvara/tobacco_project_review/main/zip_file/Bin_data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2118870 (2.0M) [application/zip]
Saving to: ‘Bin_data.zip’

Bin_data.zip        100%[===================>]   2.02M  --.-KB/s    in 0.07s   

2022-06-07 1

In [4]:
category_list = ["Burley", "Turkish", "Virginia"]
category_color = {"Burley":1, "Turkish":2, "Virginia":3}
all = []
c = []
samples = []
for category in category_list:
    for filename in os.listdir(os.path.join("Bin_data", category)):
        df = pd.read_excel(os.path.join("Bin_data", category, filename))
        samples.append(df)
        c.append(category_color[category])
        out = df.iloc[:, 1:].to_numpy().flatten("F").tolist()
        all.append(out)
        if len(out) != 11000:
            print(filename)

In [15]:
b  = [i for i,j in zip(all,c) if j == 2]
a  = [i for i,j in zip(all,c) if j == 1]


len(st.ttest_ind(a=a, b=b).pvalue)

11000

In [34]:
idxs = np.array([i for i,j in enumerate(st.ttest_ind(a=a, b=b).pvalue) if j < 0.01])

In [28]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

def test_raw_data(all, c):
    avgnural = 0
    avgrf = 0
    avglr = 0
    test_case = [0, 12, 34, 42, 75, 100, 150, 123, 343, 1034, 5643]
    highest = []
    lowest = [999999999]
    for i in test_case:
        print("random state:", i)
        X_train, X_test, y_train, y_test = train_test_split(all, c, test_size=0.33, random_state=i, stratify=c)

        reg = LogisticRegression(max_iter=100000000)
        rf = RandomForestClassifier(n_estimators=1000, random_state=0)
        clf = MLPClassifier(alpha=1e-05, hidden_layer_sizes=(20,), random_state=42,max_iter=100000,
                    solver='lbfgs')
        clf.fit(X_train, y_train)
        a = clf.score(X_test, y_test)

        rf.fit(X_train, y_train)
        b = rf.score(X_test, y_test)

        reg.fit(X_train, y_train)
        c1 = reg.score(X_test,y_test)
        print("MLP RF LR scores:", a,b,c1)
        avgnural += a
        avgrf += b
        avglr += c1
        if sum(highest)/3 < sum([a,b,c1])/3:
            highest = [a,b,c1]
        if sum(lowest)/3 > sum([a,b,c1])/3:
            lowest = [a,b,c1]
    print(
        "avg MLP: ", avgnural/len(test_case)
    )
    print(
        "avg RF: ", avgrf/len(test_case)
    )
    print(
        "avg LR: ", avglr/len(test_case)
    )
    print("highest:", highest)
    print("lowest:", lowest)
    return (avgnural/len(test_case) + avgrf/len(test_case) + avglr/len(test_case))/3

array([  204,  1179,  1463,  2084,  3201,  3601,  3861,  4203,  4602,
        5064,  5202,  5269,  5402,  5985,  6189,  6201,  6585,  6801,
        7202,  7407,  7601,  8666,  9202,  9381, 10718, 10839])

In [35]:
data = all.copy()
out = []
for row in data:
    out.append([row[i] for i in idxs])


In [36]:
test_raw_data(out, c)

random state: 0
MLP RF LR scores: 0.4444444444444444 0.48148148148148145 0.4444444444444444
random state: 12
MLP RF LR scores: 0.4074074074074074 0.4444444444444444 0.4444444444444444
random state: 34
MLP RF LR scores: 0.4074074074074074 0.4074074074074074 0.4444444444444444
random state: 42
MLP RF LR scores: 0.48148148148148145 0.4074074074074074 0.4444444444444444
random state: 75
MLP RF LR scores: 0.48148148148148145 0.4444444444444444 0.4444444444444444
random state: 100
MLP RF LR scores: 0.4074074074074074 0.4074074074074074 0.4444444444444444
random state: 150
MLP RF LR scores: 0.5185185185185185 0.48148148148148145 0.4444444444444444
random state: 123
MLP RF LR scores: 0.37037037037037035 0.3333333333333333 0.4074074074074074
random state: 343
MLP RF LR scores: 0.2962962962962963 0.25925925925925924 0.4444444444444444
random state: 1034
MLP RF LR scores: 0.5185185185185185 0.5185185185185185 0.4444444444444444
random state: 5643
MLP RF LR scores: 0.4444444444444444 0.40740740740

0.430976430976431

In [39]:
X = pd.DataFrame(all, columns=[i for i in range(11000)])

In [40]:
X

,0,1,2,3,4,5,6,7,8,9,...,10990,10991,10992,10993,10994,10995,10996,10997,10998,10999
0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.109594,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.00000,0.020773,0.0,0.000000,0.000000,0.074223,0.0,...,0.309668,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000
3,1.001617,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.784711,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000
4,0.805899,1.398739,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,9.440707,0.0,0.000000,0.00000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000
75,0.000000,0.619845,0.000000,0.16677,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,7.505323,0.000000,0.000000,0.0,2.366447,0.00000,0.000000,0.000000
76,0.000000,0.000000,0.000000,0.00000,0.233617,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,2.312844,1.228830,0.000000,0.0,0.000000,0.40784,0.000000,6.463195
77,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.105585,0.000000,0.140761,0.000000,0.0,0.079113,0.00000,0.051187,0.000000


In [41]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.datasets import load_boston


y = c
x_features = sm.add_constant(X)
ols_model = sm.OLS(y, x_features)
fit_results = ols_model.fit()
print(fit_results.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.760
Model:                            OLS   Adj. R-squared:                 -1.674
Method:                 Least Squares   F-statistic:                    0.3122
Date:                Tue, 07 Jun 2022   Prob (F-statistic):              0.995
Time:                        10:51:37   Log-Likelihood:                -41.321
No. Observations:                  79   AIC:                             226.6
Df Residuals:                       7   BIC:                             397.2
Df Model:                          71                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.317e+10    1.3e+10      1.017      0.3